In [40]:
from pennylane import numpy as np
import pennylane as qml
from pennylane import qchem
from matplotlib import pyplot as plt
#from functions_vqe_cnot import vqe_hee
#from functions_vqe_th import vqe_ng
from functions_vqe_crx_2 import vqe_hee, vqe_ng
import numpy as np
from joblib import Parallel, delayed

device = 'cpu'

## He<sub>2</sub> hamiltonian definition 

In [41]:
dataset = qml.data.load('qchem', molname="He2")[0]
H, qubits = dataset.hamiltonian, len(dataset.hamiltonian.wires)
print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  8
The Hamiltonian is  -1.6743100655776797 * I([0, 1, 2, 3, 4, 5, 6, 7]) + 0.3416412951109973 * Z(0) + 0.01523407338776362 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 0.01523407338776362 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.3416408311620236 * Z(2) + 0.025441159889832372 * (Z(0) @ Z(2)) + 0.015234388911867857 * (Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.015234388911867857 * (X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + 7.317214998671384e-08 * (Z(0) @ Y(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 7.317214998671384e-08 * (Z(0) @ X(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.5193693204313432 * Z(4) + 0.09152839641299164 * (Z(0) @ Z(4)) + -0.5193733761290926 * Z(6) + 0.0915284611478574 * (Z(0) @ Z(6)) + 0.34164129511099733 * Z(1) + 0.14108387203893755 * (Z(0) @ Z(1)) + 0.03956129782205163 * (Y(0) @ Z(2) @ Z(3) @ Y(4)) + 0.03956129782205163 * (X(0) @ Z(2) @ Z(3) @ X(4)) + 0.11564281616296 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.11564281616296 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.11564281616296 * (X(0) @ X(1) @

In [42]:
dev = qml.device("lightning.qubit", wires=qubits)

In [43]:
electrons = 4
hf_state = qml.qchem.hf_state(electrons, qubits)
singles, doubles = qchem.excitations(electrons, qubits)
print(hf_state)

[1 1 1 1 0 0 0 0]


In [44]:
s_wires, d_wires = qml.qchem.excitations_to_wires(singles, doubles)

## Distribution of results

### Parameters

In [46]:
max_iterations = 200

layer = 20

n_runs = 50

conv_tol = 1e-06

weights_lr_list = np.linspace(0.1, 0.0001, 5)
phi_lr_list = np.linspace(0.1, 0.0001, 5)
n_weights = len(phi_lr_list)

theta = 20

In [47]:
prefix1 = "../local_data/log15/"
spec1 = "_crx_1"

prefix2 = prefix1
spec2 = "_crx_2"

In [48]:
import json

run_info = {
    "max_iterations": max_iterations,
    "layers": layer,
    "n_runs": n_runs,
    "conv_tol": conv_tol,
    "weights_lr": weights_lr_list.tolist(),
    "phi_lr": phi_lr_list.tolist(),
    "prefix1": prefix1,
    "spec1": spec1,
    "prefix2": prefix2,
    "spec2": spec2,
    "device": device,
    "qubits": qubits,
    "thetas": theta
}

with open(prefix1 + "run_info.txt", "w") as f:
    json.dump(run_info, f, indent=4)

In [49]:
def run_single_simulation_2(i, k, j, weights_lr, phi_lr):
    """
    Worker function that runs one iteration.
    Arguments are passed directly, not as a tuple.
    """
    energy, grad_norm, grad_var, conv = vqe_ng(H, qubits, layer, theta=theta, weights_lr=weights_lr, phi_lr=phi_lr, max_iterations=max_iterations, conv_tol=1e-06, verbose=False)

    return i, k, j, (energy, grad_norm, grad_var, conv)

### NG

In [50]:
tasks = (delayed(run_single_simulation_2)(i, k, j, weights_lr, phi_lr)
         for i, weights_lr in enumerate(weights_lr_list)
         for k, phi_lr in enumerate(phi_lr_list)
         for j in range(n_runs))
print('number of tasks:', n_weights**2 * n_runs)
results = Parallel(n_jobs=20, verbose=11)(tasks)

number of tasks: 1250


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:   26.2s
[Parallel(n_jobs=20)]: Done   2 tasks      | elapsed:   26.9s
[Parallel(n_jobs=20)]: Done   3 tasks      | elapsed:   27.0s
[Parallel(n_jobs=20)]: Done   4 tasks      | elapsed:   27.3s
[Parallel(n_jobs=20)]: Done   5 tasks      | elapsed:   29.5s
[Parallel(n_jobs=20)]: Done   6 tasks      | elapsed:   29.9s
[Parallel(n_jobs=20)]: Done   7 tasks      | elapsed:   30.2s
[Parallel(n_jobs=20)]: Done   8 tasks      | elapsed:   30.5s
[Parallel(n_jobs=20)]: Done   9 tasks      | elapsed:   30.9s
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:   31.1s
[Parallel(n_jobs=20)]: Done  11 tasks      | elapsed:   31.7s
[Parallel(n_jobs=20)]: Done  12 tasks      | elapsed:   31.7s
[Parallel(n_jobs=20)]: Done  13 tasks      | elapsed:   31.9s
[Parallel(n_jobs=20)]: Done  14 tasks      | elapsed:   32.3s
[Parallel(n_jobs=20)]: Done  15 tasks      | elapsed:  

#### Save

In [63]:
results[0][3]

([-1.7399711608886719,
  -1.7399711608886719,
  -1.6930233240127563,
  -1.8221505880355835,
  -1.9888036251068115,
  -2.275453805923462,
  -2.486751079559326,
  -2.752943754196167,
  -2.949849843978882,
  -3.1127243041992188,
  -3.225625514984131,
  -3.341663122177124,
  -3.5580286979675293,
  -3.682323455810547,
  -3.7690792083740234,
  -3.87648344039917,
  -4.012090682983398,
  -4.182509899139404,
  -4.318049430847168,
  -4.397902965545654,
  -4.499176502227783,
  -4.617805004119873,
  -4.720419406890869,
  -4.7394208908081055,
  -4.7676100730896,
  -4.794076442718506,
  -4.801336288452148,
  -4.703168869018555,
  -4.748439311981201,
  -4.976044654846191,
  -4.959655284881592,
  -4.9491286277771,
  -5.114476203918457,
  -5.050688743591309,
  -5.175238132476807,
  -5.22823429107666,
  -5.2112226486206055,
  -5.322087287902832,
  -5.272744178771973,
  -5.365971088409424,
  -5.363377094268799,
  -5.374626636505127,
  -5.436941146850586,
  -5.419737339019775,
  -5.459100246429443,
  -5.4

In [64]:
# 1. Initialize result arrays (same as before)
energies = np.zeros((n_weights, n_weights, n_runs, max_iterations+1))
grad_norms = np.zeros((n_weights, n_weights, n_runs, max_iterations))
grad_variances = np.zeros((n_weights, n_weights, n_runs, max_iterations))
convergences = np.zeros((n_weights, n_weights, n_runs), dtype=bool)

for i, k, j, result in results:
    energies[i, k, j, :] = np.array(result[0])
    grad_norms[i, k, j, :] = np.array(result[1])
    grad_variances[i, k, j, :] = np.array(result[2])
    convergences[i, k, j] = result[3]

In [65]:
np.save(prefix2+"energies"+spec2+".npy", energies)
np.save(prefix2+"grad_norms"+spec2+".npy", grad_norms)
np.save(prefix2+"grad_variances"+spec2+".npy", grad_variances)
np.save(prefix2+"convergences"+spec2+".npy",convergences)